### 0. Requirements

In [32]:
import re
import os
import pandas as pd

### 1. 컬럼 선택

- 필요 컬럼: 일자, 언론사, 기고자, 제목, 키워드, 특성추출(가중치순 상위 50개), 본문, URL
- 제거 대상 컬럼: 뉴스 식별자, 일자, 언론사, 기고자, 제목, 통합 분류1, 통합 분류2, 통합 분류3, 사건/사고 분류1, 사건/사고 분류2, <br> 사건/사고 분류3, 인물, 위치, 기관, 키워드, 특성추출(가중치순 상위 50개), 본문, URL, 분석제외 여부

In [33]:
# 현재 스크립트가 실행되는 폴더를 기준으로 'News' 폴더 설정
BASE_DIR = os.getcwd()  # 현재 실행 중인 디렉토리 (어떤 기기에서도 동일 적용 가능)
DATA_DIR = os.path.join(BASE_DIR, "News")  # 'News' 폴더를 자동으로 설정

In [34]:
# 전처리 할 파일 리스트
file_list = [
    os.path.join(DATA_DIR, "BIGKINDSNews_20241103-20241202.xlsx"),
    os.path.join(DATA_DIR, "BIGKINDSNews_20241203-20250102.xlsx"),
    os.path.join(DATA_DIR, "BIGKINDSNews_20250103-20250203.xlsx")
]

# 여러 개의 파일을 불러와 하나의 DataFrame으로 합치기
df_list = []
for file in file_list:
    if os.path.exists(file):  # 파일이 존재하는지 확인
        df_list.append(pd.read_excel(file))
    else:
        print(f"파일 없음: {file}")

if not df_list:
    raise FileNotFoundError("전처리할 파일이 존재하지 않습니다.")

df = pd.concat(df_list, ignore_index=True) # ignore_index=True: 기존 index 무시, 새로운 연속된 index 생성

print("파일 로드 및 병합 완료.")

c:\Users\Playdata\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Playdata\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Playdata\miniconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


파일 로드 및 병합 완료.


In [35]:
# 필요한 컬럼만 선택 후 df 생성
columns_needed = ['일자', '언론사', '기고자', '제목', '키워드', '특성추출(가중치순 상위 50개)', '본문', 'URL']
df = df[columns_needed]

### 2. 날짜 형식 변환

- 문자열(YYYY-MM-DD) 형식을 datetime 형식으로 변환하여 날짜별 분석이 가능하도록 수정

In [36]:
# '일자' 컬럼을 YYYY-MM-DD 형식으로 변환
df['일자'] = df['일자'].astype(str).str[:8]  # YYYYMMDD 형식 유지
df['일자'] = pd.to_datetime(df['일자'], format="%Y%m%d", errors="coerce")
df['일자'] = df['일자'].dt.strftime("%Y-%m-%d")  # 날짜만 남기기

### 3. 중복 데이터 제거

- 기준: 제목, 본문
- 중복 제거 후 인덱스 정리(재정렬)

In [37]:
def remove_duplicates(df):
    """
    제목과 본문이 동일한 경우 중복으로 제거하고, 인덱스를 재정렬하는 함수

    Parameters:
        pd (pd.DataFrame): 데이터프레임

    Returns:
        pd.DataFrame: 중복 제거 및 인덱스 정리된 데이터프레임
    """
    df = df.drop_duplicates(subset=['제목', '본문']).reset_index(drop=True)
    return df

# 중복 제거 적용
df = remove_duplicates(df)

### 4. 결측값(NaN) 처리

In [38]:
# '일자', '제목', '본문', 'URL' 중 하나라도 결측값이 있는 행 제거
df.dropna(subset=['일자', '제목', '본문', 'URL'], inplace=True) # inplace=True: 원본 df 직접 수정

# 제거 후 인덱스 재정렬
df.reset_index(drop=True, inplace=True)

In [39]:
# 결측값 개수 확인
print(df[['일자', '제목', '본문', 'URL']].isnull().sum())

일자     0
제목     0
본문     0
URL    0
dtype: int64


### 5. 날짜 기준 최신순 정렬

In [40]:
# 최신순 정렬 후 인덱스 재정렬
df.sort_values(by='일자', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)

### 6. 텍스트 전처리

- 텍스트 데이터에서 불필요한 기호, 특수문자, 공백, 이모지 제거

In [41]:
# 텍스트 정제 함수 정의
def clean_text(text):
    if not isinstance(text, str):
        return ""
    # 연속된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 특수문자 제거(한글, 영어, 숫자만 남기기)
    text = re.sub(r'[^\w\s가-힣]', '', text)
    # 이모지 제거(유니코드 범위 내)
    text = re.sub(r'[\U00010000-\U0010FFFF]', '', text)
    return text.strip()

# 제목과 본문 컬럼에 적용
df['제목'] = df['제목'].apply(clean_text)
df['본문'] = df['본문'].apply(clean_text)

### 7. 키워드 및 특성추출 컬럼 변환

- BIGKINDS 제공 데이터: 키워드, 특성추출(가중치순 상위 50개)
- 리스트 형태의 컬럼을 KPF-BERT에서 활용하기 위해 쉼표(,)로 구분된 문자열로 변환

In [42]:
def list_to_str(x):
    if pd.isna(x):  # 결측값 처리
        return ""
    # 리스트일 경우 쉼표로 구분하여 문자열 변환
    elif isinstance(x, list):
        return ', '.join(map(str, x))
    # 리스트가 아닐 경우 문자열 변환
    else:
        return str(x)

df['키워드'] = df['키워드'].apply(list_to_str)
df['특성추출(가중치순 상위 50개)'] = df['특성추출(가중치순 상위 50개)'].apply(list_to_str)


### 8. 최종 정리 및 저장

- CSV 파일로 저장하여 KPF-BERT 모델 학습 및 추론에 사용할 수 있도록

In [43]:
# 저장 경로 설정 및 압축 파일로 저장
save_path = os.path.join(DATA_DIR, "BIGKINDS_cleaned.csv")

# 디렉토리 존재 여부 확인 후 생성
os.makedirs(DATA_DIR, exist_ok=True)

In [44]:
# 전처리된 데이터 저장
df.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"파일 저장 완료: {os.path.abspath(save_path)}")

파일 저장 완료: c:\Users\Playdata\Desktop\final\News\BIGKINDS_cleaned.csv
